In [1]:
import pandas as pd
import glob
import json


In [2]:
folders = glob.glob("Data/*")

In [3]:
dictlist = []

for folder in folders:
    if folder == "Data/user":
        continue
    files = glob.glob(folder + "/tweets/*")
    for file in files:
        json_string = open(file,'r',encoding="utf8").read()
        json_dict = json.loads(json_string)
        json_dict["search_term"] = folder[5:].capitalize()
        dictlist.append(json_dict)

In [4]:
df = pd.DataFrame(dictlist)
df = df.replace({'\n': ' '}, regex=True) # remove linebreaks in the dataframe
df = df.replace({'\t': ' '}, regex=True) # remove tabs in the dataframe
df = df.replace({'\r': ' '}, regex=True) # remove carriage return in the dataframe

In [5]:
df.sort_values("datetime",ascending = True,inplace=True,na_position="last")

In [6]:
df

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,is_reply,is_retweet,user_id,search_term,has_media,medias
2343,wikirandom,12870169175,Early thermal weapons http://en.wikipedia.org...,/wikirandom/status/12870169175,0,0,0,2010-04-26 13:00:04,False,False,124412057,early thermal weapons,NaN,NaN
2347,wiki_bot,42592438874943488,http://bit.ly/fBN9zg Early thermal weapons #...,/wiki_bot/status/42592438874943488,0,0,0,2011-03-01 20:00:03,False,False,122383622,early thermal weapons,NaN,NaN
3724,BeerBanzai,66836933933928448,(CONTD. FROM PREVIOUS) → Therefore we must str...,/BeerBanzai/status/66836933933928448,0,0,0,2011-05-07 17:39:01,False,False,92426796,interplate earthquake,NaN,NaN
2352,AlmanaraMedia,79645490265866240,@Sarahdrah this is about the early thermal we...,/AlmanaraMedia/status/79645490265866240,0,0,1,2011-06-12 01:55:39,True,False,99856628,early thermal weapons,NaN,NaN
2344,CluG1rleso,157775084457701376,"Early thermal weapons : Mining (military), Sie...",/CluG1rleso/status/157775084457701376,0,0,0,2012-01-13 16:14:46,False,False,430831584,early thermal weapons,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399,eq_map_ww2,1290611335395528705,"【M4.7】NEAR EAST COAST OF HONSHU, JAPAN Depth 8...",/eq_map_ww2/status/1290611335395528705,0,0,0,2020-08-04 17:01:35,False,False,1144740888,earthquakes,True,[https://t.co/chXLWuXpWF]
3437,triplejplays,1290611335248531458,.@HockeyDadband - Heavy Assault [21:31],/triplejplays/status/1290611335248531458,0,0,0,2020-08-04 17:01:35,False,False,86848460,heavy,NaN,NaN
214,JstLangevin,1290611335970148352,Mais nos autorites n’ont pas l’ air de s’en r...,/JstLangevin/status/1290611335970148352,0,0,0,2020-08-04 17:01:35,False,False,1241669749,air,NaN,NaN
3765,jtnpfNIl0QSLfKM,1290611334392852480,入れさせてもらった…(ニッコリ),/jtnpfNIl0QSLfKM/status/1290611334392852480,0,0,0,2020-08-04 17:01:35,True,False,1290063380019998721,jupiter,NaN,NaN


In [7]:
with open("CollectedData.csv" , "w", encoding="utf-8") as f:
    df.to_csv(f,index=False)

In [38]:
import requests , bs4,urllib
df = pd.DataFrame(pd.read_csv("CollectedData.csv"))
image_df = df[["ID","has_media","medias","search_term"]]
image_df.dropna(inplace=True)
indexes = image_df[image_df["medias"] == "[]"].index
image_df.drop(indexes,inplace=True)
image_df.head()

,ID,has_media,medias,search_term
94,1010193501798191104,True,['https://t.co/CyP4LtCbiP'],zipingpu dam
97,1016774079611797504,True,['https://t.co/BA8THZOQwG'],zipingpu dam
104,1060656032219840512,True,['https://t.co/b1E3yOprAO'],yellowstone fires of 1988
109,1075261237246783488,True,['https://t.co/DFSXZBSBNg'],yellowstone fires of 1988
115,1081830923023368192,True,['https://t.co/gA1izMnxV0'],ocean surface wave


In [39]:
import mimetypes,os,os.path

def is_url_image(url):    
    mimetype,encoding = mimetypes.guess_type(url)
    print((mimetype and mimetype.startswith('image')))
    return (mimetype and mimetype.startswith('image'))

In [41]:
def download_image(item):
    print(item)
    import requests
    import time
    if item["medias"] == None:
        return 
    item["medias"] = item["medias"].strip("][")
    item["medias"] = item["medias"].strip("\'")
    
    r = requests.get(item["medias"], stream=True)
    if os.path.isdir("Data/{}/content".format(item["search_term"])) == False:
        os.mkdir("Data/{}/content".format(item["search_term"]))
    ext = r.headers['content-type'].split('/')[-1]
    ext = ext.split(";")[0] # converts response headers mime type to an extension (may not work with everything)
    with open("Data/{}/content/{}.{}".format(item["search_term"],item["ID"],ext), 'wb') as f: # open the file to write as binary - replace 'wb' with 'w' for text files
        for chunk in r.iter_content(1024): # iterate on stream using 1KB packets
            f.write(chunk) # write the file


image_df = image_df.apply(download_image , 1)

232, dtype: object
ID                     1215094309801742336
has_media                             True
medias         ['https://t.co/NOBcg6UNNK']
search_term     european heat wave of 2003
Name: 233, dtype: object
ID                     1215094309801742336
has_media                             True
medias         ['https://t.co/NOBcg6UNNK']
search_term          extensional tectonics
Name: 234, dtype: object
ID                     1215094309801742336
has_media                             True
medias         ['https://t.co/NOBcg6UNNK']
search_term           extreme value theory
Name: 235, dtype: object
ID                     1215094309801742336
has_media                             True
medias         ['https://t.co/NOBcg6UNNK']
search_term                      be bother
Name: 236, dtype: object
ID                     1215094309801742336
has_media                             True
medias         ['https://t.co/NOBcg6UNNK']
search_term                        damaged
Name: 237, dtype: obj

SSLError: (MaxRetryError('HTTPSConnectionPool(host=\'doi.org\', port=443): Max retries exceeded with url: /10.21105/joss.02083 (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, \'WSAECONNRESET\')")))'), 'occurred at index 379')